In [357]:
import pandas as pd
import geopandas as gp
import numpy as np
import os
import itertools
import glob
from datetime import date

In [358]:
## initiate file names for output results
current_results_file = f"nov23_results_joined_SL_{date.today().strftime('%Y%m%d')}"
all_results_file = f"all_results_joined_SL_{date.today().strftime('%Y%m%d')}"

# initate folder paths for data to read in
folder_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12"
prev_results = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/all_results_joined_SL_20231214.csv"
sl = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/Master_Screening_Levels.xlsx"

In [390]:
file_extension = '*.xlsx'

# List all files with the specified extension in the folder
files = glob.glob(os.path.join(folder_path, file_extension))

results_df = []
# Iterate through each file and read its content
for file_path in files:
    print(file_path)
    df = pd.read_excel(file_path)
    try:
        print(df['Field_Collection_Start_Date'].unique())
    except:
        print(df['Field Collection Start Date'].unique())
    results_df.append(df)

results_df = pd.concat(results_df)
print(len(results_df))
results_df['Field Collection Start Date'].unique()

/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/311115 SGS EDD.xlsx
<DatetimeArray>
['NaT', '2023-11-07 00:00:00']
Length: 2, dtype: datetime64[ns]
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/311175 SGS EDD Soil.xlsx
<DatetimeArray>
['NaT', '2023-11-08 00:00:00']
Length: 2, dtype: datetime64[ns]
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/311175 SGS EDD water.xlsx
<DatetimeArray>
['NaT', '2023-11-08 00:00:00']
Length: 2, dtype: datetime64[ns]
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/B8755_EDD.xlsx
<DatetimeArray>
['NaT', '2023-11-07 00:00:00']
Length: 2, dtype: datetime64[ns]
/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/2023 Screening Results/2023.12/University of Washington Bothell Duwamish RCN 311018 EIM.xlsx
<DatetimeArray>
['2023-11-01 00:00:00']
Length: 1, dtype: datetime6

<DatetimeArray>
['NaT', '2023-11-01 00:00:00', '2023-11-17 00:00:00', '2023-11-08 00:00:00']
Length: 4, dtype: datetime64[ns]

In [ ]:
# remove any rows without a sample ID
results_df =  results_df[results_df['Sample ID'].isna() == False]

In [ ]:
# change sample ID DPS1 -> DPS-1
results_df['Sample ID'] = np.where(results_df['Sample ID'] == 'DPS1', 'DPS-1', results_df['Sample ID'])

# change sample ID SPB-0159-S-1 -> SPB-O159-S-1
results_df['Sample ID'] = np.where(results_df['Sample ID'] == 'SPB-0159-S-1', 'SPB-O159-S-1', results_df['Sample ID'])

# remove trailing spaces in sample id
results_df['Sample ID'] = results_df['Sample ID'].str.strip()


In [ ]:
results_df['Field Collection Start Date'].unique()

<DatetimeArray>
['2023-11-01 00:00:00', '2023-11-17 00:00:00', '2023-11-08 00:00:00']
Length: 3, dtype: datetime64[ns]

JOIN TABLES OF RESULTS TO MASTER SCREENING LEVELS FROM F&B

In [ ]:
# bring in screening results and master screening level spreadsheets
S4_S6_results = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/S4 and S6 Screening Results_CLEAN.xlsx"

In [ ]:
# create data frames of the raw data and the lookup
S4_S6_results_df = pd.read_excel(S4_S6_results, sheet_name='RAW_DATA')
pcb_arc_lookup = pd.read_excel(S4_S6_results, sheet_name='PCB to Aroclor Lookup')

# create data frame of the screening levels for soils and
sl_soil_df = pd.read_excel(sl, sheet_name='Soil')
sl_water_df = pd.read_excel(sl, sheet_name='Water')

In [ ]:
# format pcb to aroclor loo
pcb_arc_lookup = pcb_arc_lookup[pcb_arc_lookup['Aroclor Name'].str.contains('aroclor', na=False)][['PCB Isomer', 'Aroclor Name']]

In [ ]:
# replace pcb names with the aroclor names to match with F&B results
sl_soil_df_join = pd.merge(sl_soil_df, pcb_arc_lookup, how = 'outer', left_on = 'Chemical', right_on = 'PCB Isomer')
sl_soil_df_join['Chemical'] = np.where(sl_soil_df_join['Aroclor Name'].str.contains('aroclor', na=False), sl_soil_df_join['Aroclor Name'], sl_soil_df_join['Chemical'])
sl_soil_df_join.drop(columns = pcb_arc_lookup.columns, inplace = True)
sl_soil_df_join = sl_soil_df_join[sl_soil_df_join['Medium']=='Soil']

In [ ]:
# replace pcb names with the aroclor names to match with F&B results
sl_water_df_join = pd.merge(sl_water_df, pcb_arc_lookup, how = 'outer', left_on = 'Chemical', right_on = 'PCB Isomer')
sl_water_df_join['Chemical'] = np.where(sl_water_df_join['Aroclor Name'].str.contains('aroclor', na=False), sl_water_df_join['Aroclor Name'], sl_water_df_join['Chemical'])
sl_water_df_join.drop(columns = pcb_arc_lookup.columns, inplace = True)
sl_water_df_join = sl_water_df_join[sl_water_df_join['Medium']=='Water']

In [ ]:
# remove unnecessary columns from raw data
results_df = results_df[['Sample ID','Field Collection Start Date','Sample Matrix','Sample Source','Result Parameter Name','Result Value',
       'Result Value Units', 'Result Reporting Limit',
       'Result Reporting Limit Type', 'Result Detection Limit',
       'Result Detection Limit Type', 'Result Data Qualifier']]

In [ ]:
results_df['Field Collection Start Date'].unique()

<DatetimeArray>
['2023-11-01 00:00:00', '2023-11-17 00:00:00', '2023-11-08 00:00:00']
Length: 3, dtype: datetime64[ns]

In [ ]:
# split results into soil and water
results_soil_df = results_df[results_df['Sample Source']=='Soil']
results_water_df = results_df[results_df['Sample Matrix']=='Water']

In [ ]:
results_soil_df['Field Collection Start Date'].unique()

<DatetimeArray>
['2023-11-01 00:00:00', '2023-11-17 00:00:00', '2023-11-08 00:00:00']
Length: 3, dtype: datetime64[ns]

In [ ]:
# join screening levels to the results
soil_sl_join = pd.merge(sl_soil_df_join,results_soil_df,how = 'outer', left_on = ['Chemical'], right_on = ['Result Parameter Name'])

In [ ]:
soil_sl_join['Field Collection Start Date'].unique()

<DatetimeArray>
['2023-11-01 00:00:00', '2023-11-17 00:00:00', '2023-11-08 00:00:00', 'NaT']
Length: 4, dtype: datetime64[ns]

In [ ]:
# remove screening levels that do not have values
soil_sl_join = soil_sl_join[(soil_sl_join['Screening Level Measurement']!='na')]
soil_sl_join = soil_sl_join[(soil_sl_join['Screening Level Measurement']!='TBD')]
soil_sl_join = soil_sl_join[(soil_sl_join['Screening Level Measurement']!='PQL')]
soil_sl_join['Screening Level Measurement'].astype(float)

0       0.77
1       0.77
2       0.77
3       0.77
4       0.77
        ... 
3105     NaN
3106     NaN
3107     NaN
3108     NaN
3109     NaN
Name: Screening Level Measurement, Length: 2413, dtype: float64

In [ ]:
# calculate whether the screening levels have been exceeded
soil_sl_join['SL_exceeded'] = np.where(soil_sl_join['Screening Level Measurement'] <soil_sl_join['Result Value'],'Y','N')

In [ ]:
soil_sl_join['Field Collection Start Date'].unique()

<DatetimeArray>
['2023-11-01 00:00:00', '2023-11-17 00:00:00', '2023-11-08 00:00:00', 'NaT']
Length: 4, dtype: datetime64[ns]

WATER

In [ ]:
# join screening levels to results
water_sl_join = pd.merge(sl_water_df_join,results_water_df,how = 'outer', left_on = ['Chemical'], right_on = ['Result Parameter Name'])

In [ ]:
water_sl_join = water_sl_join[(water_sl_join['Screening Level Measurement']!='na')]
water_sl_join = water_sl_join[(water_sl_join['Screening Level Measurement']!='TBD')]
water_sl_join = water_sl_join[(water_sl_join['Screening Level Measurement']!='PQL')]
water_sl_join['Screening Level Measurement'].astype(float)

0       0.000052
1       0.000052
2       0.000052
3       0.000052
4       0.009700
          ...   
1813         NaN
1814         NaN
1815         NaN
1816         NaN
1817         NaN
Name: Screening Level Measurement, Length: 1691, dtype: float64

In [ ]:
# convert any screening levels that are in mg/L to ug/L
water_sl_join[water_sl_join['SL Unit'] == 'mg/L']

water_sl_join['Screening_Level_Measurement_Convert'] = np.where(water_sl_join['SL Unit'] == 'mg/L', water_sl_join['Screening Level Measurement']*1000, water_sl_join['Screening Level Measurement'])
water_sl_join['Screening_Level_Measurement_Convert'] = np.where(water_sl_join['SL Unit'] == 'ppm', water_sl_join['Screening Level Measurement']*1000, water_sl_join['Screening Level Measurement'])

In [ ]:
# calculate whether the screening levels have been exceeded
water_sl_join['SL_exceeded'] = np.where(water_sl_join['Screening_Level_Measurement_Convert']< water_sl_join['Result Value'], 'Y','N')

In [ ]:
columns = ['Medium', 'Sample ID','Field Collection Start Date', 'Chemical Group', 'Chemical', 'Result Parameter Name', 'Scenario', 'Screening Level Type',
       'Screening Level Measurement', 'Result Value', 'Result Data Qualifier', 'SL Unit', 'Source', 'SL_exceeded']

all_results = pd.concat([soil_sl_join[columns],water_sl_join[columns]])
all_results['SL_diff'] = all_results['Result Value'] - all_results['Screening Level Measurement']

In [ ]:
# where the screening level is blank, replace exceedance with "no screening level identified"
all_results['Screening Level Measurement'].fillna('No Screening Level Identified', inplace = True)
all_results['SL_exceeded'] = np.where(all_results['Screening Level Measurement']=='No Screening Level Identified','No Screening Level Identified', all_results['SL_exceeded'])

In [ ]:
all_results.dropna(subset=['Sample ID'], inplace=True)
all_results.rename(columns = {'Field Collection Start Date': 'DATE'}, inplace = True)
all_results.to_csv(f'/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/{current_results_file }.csv', index = False)

JOIN TO PREVIOUS SCREENING RESULTS

In [ ]:
# join to previous results
prev_results_df= pd.read_csv(prev_results)

In [ ]:
all_results_df = pd.concat([prev_results_df, all_results])
all_results_df.to_csv(f'/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/{all_results_file}.csv', index = False)